<a href="https://colab.research.google.com/github/vladimiralencar/DeepLearning-LANA/blob/master/DoencasDePele/Skin_Cancer_CNN_Model_CopiaGoogleDriveParaColab_v50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making A **CNN** Model !
On that kernal we built a cnn model and train it on skin-cancer dataset.
![CNN ’odel Architectural](https://image.ibb.co/n8PBkL/cover.png)

Start with importing libraries help us !

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
%matplotlib inline
import matplotlib.pyplot as plt
from glob import glob
import seaborn as sns
from PIL import Image
from sklearn.model_selection import train_test_split
import keras
from keras.utils.np_utils import to_categorical
# from keras.models import Sequential
# from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

# Nontar arquivos do google drive

In [0]:
# montar arquivos do google drive
# Load the Drive helper and mount
from google.colab import drive

# This wil4/sQGDWqag5BEQ1ZcL_AIduqhWI8gMxns8IZCr2wjTVaiI_7MmJPXGh18l prompt for authorization.
drive.mount('/content/drive', force_remount=True)

In [0]:
#!ls Dataset-Cancer-de-Pele-2019

!ls drive/'My Drive'/datasets/DoencasDePele-2019/skin-cancer-mnist-ham10000

folder = "drive/My Drive/datasets/DoencasDePele-2019/skin-cancer-mnist-ham10000"
folder = "skin-cancer-mnist-ham10000"

In [0]:
%%time
!cp drive/'My Drive'/datasets/DoencasDePele-2019/skin-cancer-mnist-ham10000/*.zip .
!ls

In [0]:
%%time
!unzip dataset-doencas-de-pele.zip
!ls

make the image path dictionary by joining the folder path from base directory `base_skin_dir` and merge the images in jpg format from both the folders `HAM10000_images_part1.zip` and `HAM10000_images_part2.zip`

In [0]:
#!ls skin-cancer-mnist-ham10000/HAM10000_images_part_2
!rm skin-cancer-mnist-ham10000/_DS_Store
!ls -ila skin-cancer-mnist-ham10000

In [0]:
#!mkdir skin-cancer-mnist-ham10000/images
!cp -r skin-cancer-mnist-ham10000/HAM10000_images_part_1/*.* skin-cancer-mnist-ham10000/images
!cp -r skin-cancer-mnist-ham10000/HAM10000_images_part_2/*.* skin-cancer-mnist-ham10000/images
!ls skin-cancer-mnist-ham10000/images

In [0]:
!ls skin-cancer-mnist-ham10000/images | wc -l

In [0]:
!ls skin-cancer-mnist-ham10000/HAM10000_images_part_2/ | wc -l

In [0]:
!ls skin-cancer-mnist-ham10000/HAM10000_images_part_1/ | wc -l

In [0]:
folder

In [0]:
folder = "skin-cancer-mnist-ham10000/images"

In [0]:
%%time
#base_skin_dir = os.path.join('.', folder)
base_skin_dir = folder
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(base_skin_dir, '*', '*.jpg'))}

lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

In [0]:
skin_dir = "skin-cancer-mnist-ham10000"
tile_df = pd.read_csv(os.path.join(skin_dir, 'HAM10000_metadata.csv'))
tile_df['path'] = tile_df['image_id'].map(imageid_path_dict.get)
tile_df['cell_type'] = tile_df['dx'].map(lesion_type_dict.get) 
tile_df['cell_type_idx'] = pd.Categorical(tile_df['cell_type']).codes
tile_df.sample(3)

## Cleaning Data !!
first thing showing if there is a `null` data.

In [0]:
len(tile_df)

In [0]:
tile_df.isnull().sum()

In [0]:
tile_df2 = tile_df.dropna()
len(tile_df2)
tile_df = tile_df2.copy()

Fill the null values by their mean.

In [0]:
tile_df['age'].fillna((tile_df['age'].mean()), inplace=True)

insure that now we have no empety data.

In [0]:
tile_df.isnull().sum()

In [0]:
import sys
print(sys.version)

## Loading and resize images !!
we resized the images to 100 * 75 whitch tensorflow can handle.

In [0]:
%%time
tile_df['image'] = tile_df['path'].map(lambda x: np.asarray(Image.open(x).resize((100,75))))

Show some image samples...

In [0]:
n_samples = 5
fig, m_axs = plt.subplots(7, n_samples, figsize = (4*n_samples, 3*7))
for n_axs, (type_name, type_rows) in zip(m_axs, 
                                         tile_df.sort_values(['cell_type']).groupby('cell_type')):
    n_axs[0].set_title(type_name)
    for c_ax, (_, c_row) in zip(n_axs, type_rows.sample(n_samples, random_state=1234).iterrows()):
        str_image = c_row['image']
        #image1 = (np.float_(str_image))
        image1 = np.asarray(str_image, dtype=np.float64, order='C')
        #image1 = np.array(str_image)
        #print(type(image1))
        #print(image1)
        c_ax.imshow(c_row['image'])
        c_ax.axis('off')
fig.savefig('category_samples.png', dpi=300)

In [0]:
# Checking the image size distribution
tile_df['image'].map(lambda x: x.shape).value_counts()

## Train Test Split !!
splitting data into train and test with 20:80 ratio.

In [0]:
%%time
features=tile_df.drop(columns=['cell_type_idx'],axis=1)
target=tile_df['cell_type_idx']

x_train_o, x_test_o, y_train_o, y_test_o = train_test_split(features, target, test_size=0.20,random_state=1234)

## Normalization !!
subtract their mean values from it and then dividing by their standard deviation.

In [0]:
%%time
x_train = np.asarray(x_train_o['image'].tolist())
x_test = np.asarray(x_test_o['image'].tolist())

x_train_mean = np.mean(x_train)
x_train_std = np.std(x_train)

x_test_mean = np.mean(x_test)
x_test_std = np.std(x_test)

x_train = (x_train - x_train_mean)/x_train_std
x_test = (x_test - x_test_mean)/x_test_std

## Label Encoding !!
Labels are 7 different classes of cell types from 0 to 6. We need to encode these lables to one hot vectors.

In [0]:
y_train = to_categorical(y_train_o, num_classes = 7)
y_test = to_categorical(y_test_o, num_classes = 7)

## Training and Validation Split !!
validate:train >> 10:90 %

In [0]:
%%time
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size = 0.1, random_state = 2)

In [0]:
%%time
x_train = x_train.reshape(x_train.shape[0], *(75, 100, 3))
x_test = x_test.reshape(x_test.shape[0], *(75, 100, 3))
x_validate = x_validate.reshape(x_validate.shape[0], *(75, 100, 3))

## Model Building !!

In [0]:
input_shape = (75, 100, 3)
num_classes = 7
model = Sequential([
    Conv2D(32, 3, padding='same', activation='relu', input_shape=input_shape),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.25),

    Conv2D(64, 3, padding='same', activation='relu', input_shape=input_shape),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.4),
    
    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.5),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.55),
    Dense(7, activation='softmax')
])

In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [0]:
model.summary()

**Data Augmentation** to avoiding the overfitting.

In [0]:
%%time
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images
datagen.fit(x_train)

## Fitting The model !!

In [0]:
%%time
epochs = 50 
batch_size = 10
history = model.fit_generator(
    datagen.flow(x_train,y_train, batch_size=batch_size),
    steps_per_epoch=x_train.shape[0] // batch_size,
    epochs=epochs,
    validation_data=(x_validate,y_validate),
    validation_steps=x_validate.shape[0] // batch_size
    #,callbacks=[learning_rate_reduction]
)

## Model Evaluation !!

In [0]:
%%time
loss, accuracy = model.evaluate(x_test, y_test, verbose=1)
loss_v, accuracy_v = model.evaluate(x_validate, y_validate, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))
model.save("model.h5")

In [0]:
#1. Function to plot model's validation loss and validation accuracy
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
    axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

In [0]:
%%time
plot_model_history(history)

# # Predição

In [0]:
from google.colab import files
files.upload()

In [0]:
x_pred.shape

In [0]:
input_shape = (75, 100, 3)

#x_pred = np.asarray(Image.open(x_pred)).resize(input_shape)

#x_pred = x_pred.reshape(input_shape)


In [0]:
x_pred.shape

In [0]:
 %%time
 #.resize((100,75))
 input_shape = (75, 100, 3)
 x = 'df.jpg'
 x_pred = np.asarray(Image.open(x).resize((100,75)))  

#x_pred = x_pred.resize(input_shape)
x_pred = x_pred.reshape(1, 75, 100, 3)
print(x_pred.shape)


In [0]:
print(model.predict(x_pred))
print(model.predict_proba(x_pred))

In [0]:
======

In [0]:
%%time

from google.colab import files
xpred = files.upload()



x_test = np.asarray(x_test_o['image'].tolist())

x_test_mean = np.mean(x_test)
x_test_std = np.std(x_test)

x_test = (x_test - x_test_mean)/x_test_std


# reshape do dado
x_test = x_test.reshape(x_test.shape[0], *(75, 100, 3))
